In [6]:
import ollama
def classify_english_question_using_gemma(english_question):
    
    prompt = f"""You are an expert question analyst. Your task is to classify the following English question as either SINGLEHOP or MULTIHOP.

Definitions:
- A question is **SINGLEHOP** if it can be answered using a single fact, sentence, or direct piece of information. Even if the question seems long, if it refers to only one piece of knowledge, it's SINGLEHOP.
- A question is **MULTIHOP** if it requires combining two or more distinct facts, reasoning through multiple steps, or connecting information from different sources.

🚨 Key Indicators:
- MULTIHOP questions often have two parts or require using the answer to one part to answer the next.
- Look for compound instructions, temporal reasoning, or layered dependencies.

✅ Examples:

1. SINGLEHOP: "What is the capital of Japan?" → Can be answered with one direct fact.
2. MULTIHOP: "What is the capital of the country that hosted the 2016 Olympics?" → First identify the country, then find its capital.

---

Now classify the following question:

Question: "{english_question}"
Answer:"""

    try:
        response = ollama.generate(
            model='gemma:7b-instruct',
            prompt=prompt
        )
        reply = response['response'].strip().lower()

        # Normalize and return classification
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {english_question}\n↪ {e}")
        return "error"


In [7]:
# Dictionary of 50 classification test questions with labels
classification_test_data = {
    # SINGLEHOP questions (20)
    "What is the capital of Canada?": "singlehop",
    "Who invented the lightbulb?": "singlehop",
    "When did World War I begin?": "singlehop",
    "What gas do humans exhale?": "singlehop",
    "What is the boiling point of water in Celsius?": "singlehop",
    "Which country is famous for the Eiffel Tower?": "singlehop",
    "Who is the current Prime Minister of the UK?": "singlehop",
    "What is the chemical symbol for gold?": "singlehop",
    "Who painted the Last Supper?": "singlehop",
    "How many legs does a spider have?": "singlehop",
    "What is the national language of Pakistan?": "singlehop",
    "Who directed the movie 'Titanic'?": "singlehop",
    "What is the name of Earth's only natural satellite?": "singlehop",
    "What is the square root of 144?": "singlehop",
    "Which continent is Egypt located in?": "singlehop",
    "What is the primary ingredient in sushi?": "singlehop",
    "What currency is used in Germany?": "singlehop",
    "Who wrote 'Pride and Prejudice'?": "singlehop",
    "How many hours are there in a day?": "singlehop",
    "What is the largest ocean on Earth?": "singlehop",

    # MULTIHOP questions (30)
    "Who was president of the US when the Berlin Wall fell, and what was his political party?": "multihop",
    "Which country won the first FIFA World Cup, and who scored the winning goal?": "multihop",
    "What is the capital of the country that borders both the Atlantic and Mediterranean Seas?": "multihop",
    "Name the author of 'The Hobbit' and identify the year it was first published.": "multihop",
    "Which European country produces the most wine, and what are its top export markets?": "multihop",
    "What is the largest island in the world, and under which country's territory does it fall?": "multihop",
    "Who was the British monarch during the American Revolution, and how long did they reign?": "multihop",
    "What language is spoken in the country that colonized Angola?": "multihop",
    "What is the title of the book featuring Sherlock Holmes and who illustrated the first edition?": "multihop",
    "Who painted Starry Night and what mental illness did he reportedly suffer from?": "multihop",
    "Which two rivers define Mesopotamia and which ancient civilizations lived there?": "multihop",
    "Which planet has the most moons, and what are two of their names?": "multihop",
    "Who discovered penicillin and what illness was it first used to treat?": "multihop",
    "Which two cities are connected by the Trans-Siberian Railway?": "multihop",
    "Who won the Nobel Peace Prize in 2014 and what was the reason?": "multihop",
    "Which animal appears on the Australian coat of arms and why was it chosen?": "multihop",
    "What invention is credited to Alexander Graham Bell and how did it evolve over time?": "multihop",
    "Name the spacecraft that took humans to the Moon and list the crew members.": "multihop",
    "What is the primary export of Venezuela and how has its economy changed in the last decade?": "multihop",
    "Which continent has the most countries and what is the largest by area there?": "multihop",
    "What programming language was used to build the first version of Instagram and who were its founders?": "multihop",
    "What empire built Machu Picchu and which explorer discovered it?": "multihop",
    "Which two countries share the longest land border and what is its total length?": "multihop",
    "Who was the first woman in space and what country did she represent?": "multihop",
    "Which civilization built the pyramids and what was their religious belief system?": "multihop",
    "What was the cause of the Cold War and which two countries were involved?": "multihop",
    "Which artist created the statue of David and what other works is he famous for?": "multihop",
    "Who assassinated Archduke Franz Ferdinand and what conflict did it trigger?": "multihop",
    "What is the capital of the country with the largest number of official languages?": "multihop",
    "Which African country gained independence last and from whom?": "multihop"
}


In [8]:
# Assume classification_test_data dictionary is already defined

# Results storage
results = []
correct_singlehop = 0
correct_multihop = 0
total_singlehop = 0
total_multihop = 0

# Classification loop
for idx, (question, true_label) in enumerate(classification_test_data.items(), 1):
    print(f"\n🔍 Processing Query {idx}/{len(classification_test_data)}:\n➤ {question}")
    
    predicted = classify_english_question_using_gemma(question)

    results.append({
        "question": question,
        "true_label": true_label,
        "predicted_label": predicted
    })

    # Count per class
    if true_label == "singlehop":
        total_singlehop += 1
        if predicted == "singlehop":
            correct_singlehop += 1
    elif true_label == "multihop":
        total_multihop += 1
        if predicted == "multihop":
            correct_multihop += 1

# Accuracy calculations
singlehop_accuracy = correct_singlehop / total_singlehop if total_singlehop > 0 else 0
multihop_accuracy = correct_multihop / total_multihop if total_multihop > 0 else 0
overall_accuracy = (correct_singlehop + correct_multihop) / (total_singlehop + total_multihop)

# Print results
print(f"\n✅ Classification Accuracy Summary:")
print(f"- SINGLEHOP: {correct_singlehop}/{total_singlehop} ({singlehop_accuracy:.2%})")
print(f"- MULTIHOP: {correct_multihop}/{total_multihop} ({multihop_accuracy:.2%})")
print(f"- OVERALL: {(correct_singlehop + correct_multihop)}/{len(classification_test_data)} ({overall_accuracy:.2%})")

# Convert results to DataFrame
import pandas as pd
df_results = pd.DataFrame(results)

# (Optional) Save to CSV
# df_results.to_csv("classification_results.csv", index=False)



🔍 Processing Query 1/50:
➤ What is the capital of Canada?

🔍 Processing Query 2/50:
➤ Who invented the lightbulb?

🔍 Processing Query 3/50:
➤ When did World War I begin?

🔍 Processing Query 4/50:
➤ What gas do humans exhale?

🔍 Processing Query 5/50:
➤ What is the boiling point of water in Celsius?

🔍 Processing Query 6/50:
➤ Which country is famous for the Eiffel Tower?

🔍 Processing Query 7/50:
➤ Who is the current Prime Minister of the UK?

🔍 Processing Query 8/50:
➤ What is the chemical symbol for gold?

🔍 Processing Query 9/50:
➤ Who painted the Last Supper?

🔍 Processing Query 10/50:
➤ How many legs does a spider have?

🔍 Processing Query 11/50:
➤ What is the national language of Pakistan?

🔍 Processing Query 12/50:
➤ Who directed the movie 'Titanic'?

🔍 Processing Query 13/50:
➤ What is the name of Earth's only natural satellite?

🔍 Processing Query 14/50:
➤ What is the square root of 144?

🔍 Processing Query 15/50:
➤ Which continent is Egypt located in?

🔍 Processing Query 16/